In [1]:
import openai

In [2]:
from openai import OpenAI

In [3]:
client = OpenAI()

In [4]:
import os

In [6]:
#os.environ

In [7]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "is it too late to join the course?"}]

)

In [10]:
response.choices[0].message.content

"To determine whether it's too late to join a course, you'll need to consider a few factors:\n\n1. **Enrollment Deadline:** Check if the course has a specific enrollment deadline. Many courses have a cutoff date after which you can't register.\n\n2. **Start Date:** If the course has already started, find out if late registration is allowed. Some courses permit late enrollment if you can catch up.\n\n3. **Course Type:** Consider whether the course is self-paced or instructor-led. Self-paced online courses often allow you to enroll at any time, while instructor-led courses may have specific timelines.\n\n4. **Prerequisites:** Ensure you meet any prerequisite requirements before joining the course.\n\n5. **Instructor or Institution Policies:** Contact the course instructor or the institution offering the course to see if exceptions can be made for late enrollment.\n\n6. **Course Structure:** Evaluate whether you can realistically catch up on missed content if the course has already starte